In [ ]:
##!/usr/bin/env python
"""plot_spinup_time_series.py

Script plots the regionally aggregated time series for different events to evaluate the WRF-LIS-CABLE physics ensemble

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu May  9 12:04:37 AEST 2019

"""

In [ ]:
# Load packages

#from __future__ import division
import numpy as np
import math
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
from matplotlib.collections import LineCollection
import common_functions as cf


In [ ]:
# All Simulation Details

sims=["PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_8YR", "PHYS_30YR_SPIN"]
rlabels=["4YR","8YR","30YR"]
fig_name_prefix='SPINUP_TIME_SERIES_AWAP_MASKED_'

In [ ]:
# Simulation Details - grouped by PBL

sims=["PHYS_TEST_RA5_PBL5_CU2", "SOILSNOW_RA5_PBL5_CU2"]
rlabels=["SSGW","SOILSNOW"]
fig_name_prefix='HYDRO_TIME_SERIES_AWAP_MASKED_'

In [ ]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')
nlat = 152
nlon = 223

# Data directory 
datadir='/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(sims)
obs="AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())

if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
vlabels=['PR [mm $day^{-1}$]','TX [\xb0 C]','TN [\xb0 C]','$Q_{E}$ [W $m^{-2}$]']
nvar=len(vlabels)



In [ ]:
# Define events
Elabel = ["Tropics","South East Australia"]
period = ["First 365 days","First 356 days"]
indst  = [1,1]
indfi  = [365,365]
latN=[-23.43,-38.0]
latX=[-10.0,-27.0]
lonN=[110.0,139.0]
lonX=[155.0,151.0]
Rname=["TROPICS","SEA"]

nreg   = len(Elabel) 

mx = np.empty((nreg,nvar),dtype=np.float64)
mx[:,0] = [15.,20.]
mx[:,1] = [45.,45.]
mx[:,2] = [30.,30.]
mx[:,3] = [100.,100.]

mn = np.empty((nreg,nvar),dtype=np.float64)
mn[:,0] = np.repeat(0,nreg)
mn[:,1] = [15.,0.]
mn[:,2] = [0.,-5.]
mn[:,3] = [0.,0.]

mnmx = np.empty((nreg,nvar),dtype=np.float64)
mnmx[:,0] = [5.,5.]
mnmx[:,1] = np.repeat(5,nreg)
mnmx[:,2] = np.repeat(5,nreg)
mnmx[:,3] = [25.,25.,]
lspace = [10,10]


In [ ]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

# If the longitude spans [-180 180] then update to that instead it is [0 360] - better of AUS domain
#if np.min(lon) < 0.0:
#    print('Longitude values span [-180 180] will change to [0 360]')
#    for jj in range(len(lon)):
#        if lon[jj] < 0.0:
#            lon[jj]=lon[jj]+360

#latT = lat[(lat>=latN) & (lat<=latX)]
#lonT = lon[(lon>=lonN) & (lon<=lonX)]

In [ ]:
# Load observational data

filename='%s/%s_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip = file.variables['awap_precip_mask'][:]
obst2max = file.variables['awap_t2max_mask'][:]
obst2min = file.variables['awap_t2min_mask'][:]
file.close()

filename='%s/GLEAM_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls = file.variables['gleam_hfls_mask'][:]

filename='%s/WALD_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls2 = file.variables['wald_hfls_mask'][:]

file.close()

ndays = 822
sind = 213

# Truncate data to simulation period
obsprecipT = obsprecip[sind:sind+ndays,:,:]
obst2maxT = obst2max[sind:sind+ndays,:,:]
obst2minT = obst2min[sind:sind+ndays,:,:]
obshflsT = obshfls[sind:sind+ndays,:,:]
obshflsT2 = obshfls2[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma = np.ma.masked_array(obsprecipT, obsprecipT>=1.e20).filled(np.nan)
obst2maxma  = np.ma.masked_array(obst2maxT, obst2maxT>=1.e20).filled(np.nan)
obst2minma  = np.ma.masked_array(obst2minT, obst2maxT>=1.e20).filled(np.nan)
obshflsma  = np.ma.masked_array(obshflsT, obst2maxT>=1.e20).filled(np.nan)
obshflsma2  = np.ma.masked_array(obshflsT2, obst2maxT>=1.e20).filled(np.nan)


In [ ]:
# Function to plot data
def plot_ts(time,tsdata,obs2,rlabels,vlabels,mtitle,figurename,mx,mn,mnmx,lspace):
    """This function plots time series for observations and models"""

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    import string
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=500
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"

    # Define dimensions
    nmod = tsdata.shape[0] - 1 # As obs, mx and mn are included
    nvar = tsdata.shape[1]
    nt = tsdata.shape[2]
      
    # Create figure object and subplots
    fig, axarr = plt.subplots(nvar, 1, figsize=(20.0,5.0*(nvar)), squeeze=False)
    tarr = np.arange(0,nt)
    
    # Have a look at the colormaps here and decide which one you'd like:
    # http://matplotlib.org/1.2.1/examples/pylab_examples/show_colormaps.html
    colors = ["red","blue","green","orange","purple"]

    # Iterate through variables
    for vind in range(nvar):

        # Models
        for mind in range(nmod):
            axarr[vind,0].plot(tarr,tsdata[mind+1,vind,:], linewidth=4,color=colors[mind], linestyle='-', label=rlabels[mind])

        # Observations
        axarr[vind,0].plot(tarr,tsdata[0,vind,:],linestyle='-',linewidth=4,color='black',label='AGCD/GLEAM')
        if vind == nvar-1:
            axarr[vind,0].plot(tarr,obs2,linestyle=':',linewidth=4,color='black',label='vD18')

        # Add title
        axarr[vind,0].set_title('(%s)' %(string.ascii_lowercase[vind]), fontweight='bold', fontsize=20, y = 0.9, x = 0.025)
        
        # Fix Labelling
        axarr[vind,0].set_ylabel('%s' %(vlabels[vind]), fontweight = 'bold',fontsize=20)
        axarr[vind,0].set_ylim(mn[vind],mx[vind])
        axarr[vind,0].set_yticks(np.arange(mn[vind],mx[vind],mnmx[vind]))
        axarr[vind,0].set_yticklabels(np.arange(np.int(mn[vind]),np.int(mx[vind]),np.int(mnmx[vind])), fontsize=18)
        if vind < nvar-1:
            axarr[vind,0].set_xticks([],[])
        else:
            axarr[vind,0].set_xticks(tarr[::lspace])
            axarr[vind,0].set_xticklabels(time[::lspace],rotation=90,fontsize=18)

    #axarr[0,0].set_title(mtitle, fontweight = 'bold')
    legend = axarr[-1,0].legend(loc='upper center', bbox_to_anchor=(0.5,-0.375), ncol=6, fontsize=20)

    fig.tight_layout()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(figurename,bbox_extra_artists=(legend,), bbox_inches='tight')
    plt.close(fig)


In [ ]:
for rind in range(nreg):

    # Get lat/lon indices to truncate data to region of interest
    bbox = [lonN[rind],lonX[rind], latN[rind], latX[rind]]
    i0,i1,j0,j1 = cf.bbox2ij(lon2d,lat2d,bbox)
    
    # Select period and region of interest
    obs_precip_sub = obsprecipma[indst[rind]:indfi[rind],j0:j1,i0:i1]
    obs_t2max_sub = obst2maxma[indst[rind]:indfi[rind],j0:j1,i0:i1]
    obs_t2min_sub = obst2minma[indst[rind]:indfi[rind],j0:j1,i0:i1]
    obs_hfls_sub = obshflsma[indst[rind]:indfi[rind],j0:j1,i0:i1]
    obs_hfls_sub2 = obshflsma2[indst[rind]:indfi[rind],j0:j1,i0:i1]

    # Compute regionally aggregated timeseries
    obs_precip_ts = np.nanmean(obs_precip_sub,axis=(1,2))
    obs_t2max_ts = np.nanmean(obs_t2max_sub,axis=(1,2))
    obs_t2min_ts = np.nanmean(obs_t2min_sub,axis=(1,2))
    obs_hfls_ts = np.nanmean(obs_hfls_sub,axis=(1,2))
    obs_hfls_ts2 = np.nanmean(obs_hfls_sub2,axis=(1,2))
    del obs_precip_sub, obs_t2max_sub, obs_t2min_sub, obs_hfls_sub, obs_hfls_sub2
    
    nt = obs_precip_ts.shape[0]
    tsall = np.empty((nmod+1,nvar,nt),dtype=np.float64)
    tsall[0,0,:] = obs_precip_ts
    tsall[0,1,:] = obs_t2max_ts
    tsall[0,2,:] = obs_t2min_ts
    tsall[0,3,:] = obs_hfls_ts
    del obs_precip_ts,obs_t2max_ts,obs_t2min_ts, obs_hfls_ts
    
    # Loop through the experiments
    for mind,mname in enumerate(sims):

        # Load precip data
        filename='%s/%s_WRF_Precip_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        precip = file.variables['wrf_precip_mask'][:]
        file.close()

        # Load temperature data
        filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        if mind == 0:
            time = nc.chartostring(file.variables['Times'][:])
            timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
        t2max = file.variables['T2MAX'][:]
        t2min = file.variables['T2MIN'][:]
        file.close()

        # Load hfls data
        filename='%s/%s_lisout_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
        file = nc.Dataset(filename)
        hfls = file.variables['lis_hfls_mask'][:]
        file.close()
        
        # Mask data according to the landsea-mask
        lsmask3d = np.repeat(lsmask[np.newaxis],precip.shape[0],axis=0)
        precipma = np.ma.masked_where(lsmask3d==0,precip)
        t2maxma = np.ma.masked_where(lsmask3d==0,t2max)
        t2minma = np.ma.masked_where(lsmask3d==0,t2min)
        hflsma = np.ma.masked_where(lsmask3d==0,hfls)
                
        # Truncate data to region and period of interest
        precip_sub = precipma[indst[rind]:indfi[rind],j0:j1,i0:i1]
        t2max_sub = t2maxma[indst[rind]:indfi[rind],j0:j1,i0:i1]
        t2min_sub = t2minma[indst[rind]:indfi[rind],j0:j1,i0:i1]
        hfls_sub = hflsma[indst[rind]:indfi[rind],j0:j1,i0:i1]

        # Compute regionally aggregated timeseries
        tsall[mind+1,0,:] = np.nanmean(precip_sub,axis=(1,2))
        tsall[mind+1,1,:] = np.nanmean(t2max_sub,axis=(1,2))
        tsall[mind+1,2,:] = np.nanmean(t2min_sub,axis=(1,2))
        tsall[mind+1,3,:] = np.nanmean(hfls_sub,axis=(1,2))
        
        del precip, t2max, t2min, hfls
        del precipma, t2maxma, t2minma, hflsma
        del precip_sub, t2max_sub, t2min_sub, hfls_sub
    
    # Plot the time series
    figurename='%s/%s%s.png' %(fig_dir,fig_name_prefix,Rname[rind])
    figuretitle='%s %s' %(Elabel[rind],period[rind])
    plot_ts(timesplit[indst[rind]:indfi[rind]],tsall,obs_hfls_ts2,rlabels,vlabels,figuretitle,figurename,mx[rind,:],mn[rind,:],mnmx[rind,:],lspace[rind])

    del lsmask3d
    del time,timesplit, tsall
